In [ ]:
# # ========================================
# # 1) Connect to Google Drive
# # ========================================
# from google.colab import drive
# drive.mount('/content/drive')

# #MODEL_DIR = "/content/drive/MyDrive/deep_learning"


Mounted at /content/drive


In [ ]:
# ========================================
# 2) Install required libraries
# ========================================
!pip install -U transformers datasets accelerate evaluate optuna wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install evaluate

In [ ]:
!pip install -U transformers


In [ ]:
# ========================================
# 3) Import libraries
# ========================================
import os, time, copy, tempfile, inspect
import numpy as np
from pathlib import Path
import pandas as pd
import torch, torch.nn as nn
from torch.quantization import quantize_dynamic
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import os
import re
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report


In [ ]:
# Check if GPU is available and select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# === Load data ===
#test_df = pd.read_csv('/content/drive/MyDrive/deep_learning/val_processed.csv', encoding='latin1')
from pathlib import Path
import pandas as pd

# Navigate from model folder to data folder
current_dir = Path.cwd()  # models/bert-base-uncased/
models_dir = current_dir.parent  # models/
project_root = models_dir.parent  # project root
data_dir = models_dir / 'data'

# Load data
test_df = pd.read_csv(data_dir / 'test_processed.csv', encoding='latin1')

print(f"Test shape: {test_df.shape}")


In [ ]:
test_df["label"] = test_df["Sentiment"].map(label2id)

In [ ]:
def clean_for_cardiffnlp(text):
    if pd.isnull(text):
        return ""

    tokens = []
    for t in text.split(" "):
        if t.startswith("@") and len(t) > 1:
            tokens.append("@user")
        elif t.startswith("http"):
            tokens.append("http")
        else:
            tokens.append(t)
    text = " ".join(tokens)

    # Normalize common COVID variants to "covid"
    text = re.sub(r"\b(coronaviruspandemic|covid[_\s-]*2019|covid[_\s-]*19|covid2019|coronavirus2019|coronavirus|corona)\b", "covid", text, flags=re.IGNORECASE)

    # Decode HTML entities
    text = html.unescape(text)

    # Normalize whitespace and repeated punctuation (optional)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"(\.\s*){2,}", ". ", text)
    text = re.sub(r"([!?]){2,}", r"\1", text)
    text = re.sub(r"(\?\s+){2,}", "?", text)
    text = re.sub(r"(\!\s+){2,}", "!", text)

    return text


In [ ]:
BASE_CHECKPOINT = "cardiffnlp/twitter-roberta-base-sentiment-latest"
LABEL_COL = "label"
BATCH_SIZE = 128
NUM_WORKERS = 2
PIN_MEMORY = True


# ==== tokenizer + collator ====
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT, use_fast=True)
collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==== Class Dataset ====
class TextClsDataset(Dataset):
    def __init__(self, df, text_col, label_col):
        self.texts = df[text_col].astype(str).tolist()
        self.labels = df[label_col].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=128,
            return_tensors=None
        )
        enc["labels"] = self.labels[idx]
        return enc

# prepere datasets
test_ds_clean = TextClsDataset(test_df, text_col="ProcessedTweet",        label_col=LABEL_COL)

# DataLoaders
test_loader_clean = DataLoader(
    test_ds_clean,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    collate_fn=collator
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# ============ CUSTOM ROBERTA WITH DROPOUT ============
class RobertaWithDropout(nn.Module):
    """
    Custom RoBERTa model with configurable dropout
    """
    def __init__(self, model_name, num_labels, dropout_rate=0.1):
        super(RobertaWithDropout, self).__init__()

        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return type('obj', (object,), {'logits': logits})()


In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# ===== config =====
# --------- config ----------
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
#MODEL_DIR  = "/content/drive/MyDrive/deep_learning"
MODEL_DIR  =data_dir

# מודלים להערכה (רק API)
MODEL_PREPROC = {
    "HF_best_model_stage3_YR.pt": "light",
}

ORDERED_LABELS = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
LABEL2ID = {l:i for i,l in enumerate(ORDERED_LABELS)}

# ===== tokenizer + collator =====
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT, use_fast=True)
collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ===== preprocessing functions =====
def preprocess_clean(t: str) -> str:
    import re, html
    if not isinstance(t, str): return ""
    toks=[]
    for w in t.split(" "):
        if w.startswith("@") and len(w)>1: toks.append("@user")
        elif w.startswith("http"): toks.append("http")
        else: toks.append(w)
    t=" ".join(toks)
    t=re.sub(r"\b(coronaviruspandemic|covid[_\s-]*2019|covid[_\s-]*19|covid2019|coronavirus2019|coronavirus|corona)\b",
             "covid", t, flags=re.IGNORECASE)
    t=html.unescape(t)
    t=re.sub(r"\s+"," ",t).strip()
    return t

def preprocess_light(t: str) -> str:
    import re
    if not isinstance(t, str): return ""
    t=re.sub(r'https?://\S+','HTTPURL',t)
    t=re.sub(r'@\w+','@USER',t)
    t=re.sub(r'\s+',' ',t).strip()
    return t

# ===== dataset =====
class TDataset(Dataset):
    def __init__(self, texts, labels, max_len=128):
        self.texts=texts; self.labels=labels; self.max_len=max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = tokenizer(self.texts[i], truncation=True, max_length=self.max_len, return_tensors="pt")
        item = {k:v.squeeze(0) for k,v in enc.items()}
        item["labels"] = torch.tensor(self.labels[i], dtype=torch.long)
        return item

# ===== model class =====
class RobertaWithDropout(nn.Module):
    def __init__(self, model_name, num_labels=5, dropout_rate=0.2):
        super().__init__()
        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)
    def forward(self, input_ids, attention_mask=None, labels=None):
        out = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:,0,:]
        logits = self.classifier(self.dropout(cls))
        return {"logits": logits}

# ===== evaluation =====
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            ids = batch["input_ids"].to(DEVICE)
            mask = batch.get("attention_mask")
            if mask is not None:
                mask = mask.to(DEVICE)
            labels = batch["labels"].cpu().numpy()
            logits = model(ids, attention_mask=mask)["logits"]
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)
    print(classification_report(all_labels, all_preds, target_names=ORDERED_LABELS, digits=4))
    return {
        "accuracy": accuracy_score(all_labels, all_preds),
        "f1": f1_score(all_labels, all_preds, average='macro'),
        "precision": precision_score(all_labels, all_preds, average='macro'),
        "recall": recall_score(all_labels, all_preds, average='macro')
    }

# ===== run all API models =====
test_df["label"] = test_df["Sentiment"].map(LABEL2ID)

for model_file, preproc_type in MODEL_PREPROC.items():
    print(f"\n=== Evaluating {model_file} ({preproc_type}) ===")
    if preproc_type == "clean":
        texts = [preprocess_clean(x) for x in test_df["OriginalTweet"].astype(str)]
    else:
        texts = [preprocess_light(x) for x in test_df["OriginalTweet"].astype(str)]
    labels = test_df["label"].tolist()

    ds = TDataset(texts, labels)
    dl = DataLoader(ds, batch_size=128, shuffle=False, collate_fn=collator)

    model = RobertaWithDropout(BASE_CHECKPOINT, num_labels=len(ORDERED_LABELS), dropout_rate=0.2)
    model.load_state_dict(torch.load(os.path.join(MODEL_DIR, model_file), map_location=DEVICE))
    model.to(DEVICE)

    metrics = evaluate_model(model, dl)
    print(f"Acc={metrics['accuracy']:.4f}, F1={metrics['f1']:.4f}, "
          f"Precision={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}")



=== Evaluating HF_best_model_stage3_YR.pt (light) ===


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

                    precision    recall  f1-score   support

Extremely Negative     0.8929    0.8733    0.8830       592
          Negative     0.8368    0.8569    0.8467      1041
           Neutral     0.9011    0.8239    0.8608       619
          Positive     0.8240    0.8353    0.8296       947
Extremely Positive     0.8501    0.8898    0.8695       599

          accuracy                         0.8539      3798
         macro avg     0.8610    0.8558    0.8579      3798
      weighted avg     0.8549    0.8539    0.8540      3798

Acc=0.8539, F1=0.8579, Precision=0.8610, Recall=0.8558


# Post-training Dynamic Quantization

In [ ]:
DEVICE = torch.device("cpu")
# Check
print(f"Using device: {DEVICE}")

Using device: cpu


In [ ]:
# ======================
# Step 1: Robust loader for  Roberta
# ======================

import os
import copy
import tempfile
from pathlib import Path

import torch
import torch.nn as nn
from torch.quantization import quantize_dynamic

from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# ======================
# Basic configuration
# ======================
MODEL_DIR  =data_dir
MODEL_FILE  = "HF_best_model_stage3_YR.pt"
BACKBONE    = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS  = 5
DROPOUT     = 0.2
OUT_DIR     = os.path.join(MODEL_DIR, "quantized_int8")
os.makedirs(OUT_DIR, exist_ok=True)

ordered_labels = ["Extremely Negative","Negative","Neutral","Positive","Extremely Positive"]

def _state_dict_size_mb(model_or_sd) -> float:
    sd = model_or_sd if isinstance(model_or_sd, dict) else model_or_sd.state_dict()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(sd, tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

# ======================
# 1) Load FP32 model
# ======================
model_path = os.path.join(MODEL_DIR, MODEL_FILE)
model = RobertaWithDropout(BACKBONE, num_labels=NUM_LABELS, dropout_rate=DROPOUT)
state_dict = torch.load(model_path, map_location="cpu")
model.load_state_dict(state_dict, strict=True)
model.eval().cpu()

param_count = sum(p.numel() for p in model.parameters())
size_fp32   = _state_dict_size_mb(model)
print(f"Loaded {MODEL_FILE} | params={param_count:,} | state_dict={size_fp32:.2f} MB")

# ======================
# 2) Dynamic quantization on encoder only (INT8)
# ======================
q_model = copy.deepcopy(model).eval().cpu()
assert hasattr(q_model, "roberta") and hasattr(q_model.roberta, "encoder"), "Expected .roberta.encoder to exist"
q_model.roberta.encoder = quantize_dynamic(q_model.roberta.encoder, {nn.Linear}, dtype=torch.qint8)

size_int8 = _state_dict_size_mb(q_model)
print(f"State dict size: FP32={size_fp32:.2f} MB -> INT8={size_int8:.2f} MB")

# ======================
# 3) Save quantized state_dict
# ======================
save_name  = f"{Path(MODEL_FILE).stem}_quantized_int8_sd.pt"
quant_path = os.path.join(OUT_DIR, save_name)
torch.save(q_model.state_dict(), quant_path)
print(f"Quantized model saved to: {quant_path}")



Loaded HF_best_model_stage3_YR.pt | params=124,649,477 | state_dict=475.58 MB
State dict size: FP32=475.58 MB -> INT8=232.63 MB
Quantized model saved to: /content/drive/MyDrive/deep_learning/quantized_int8/HF_best_model_stage3_YR_quantized_int8_sd.pt


In [ ]:
# ======================
# 4) Optional evaluation on CPU (runs only if `test_df` exists)
# ======================
@torch.inference_mode()
def eval_int8_cpu(model, dataloader, label_names=None):
    """Evaluate model on CPU and print a per-class report."""
    model.eval()
    model.cpu()
    y_true, y_pred = [], []

    for batch in dataloader:
        labels = batch.pop("labels")
        # Ensure tensors
        batch = {k: (v if isinstance(v, torch.Tensor) else torch.tensor(v)) for k, v in batch.items()}
        outputs = model(**batch)

        # Robust logits extraction
        if hasattr(outputs, "logits"):
            logits = outputs.logits
        elif isinstance(outputs, dict) and "logits" in outputs:
            logits = outputs["logits"]
        elif isinstance(outputs, (list, tuple)):
            logits = outputs[0]
        else:
            logits = outputs  # if the model returns logits directly

        preds = torch.argmax(logits, dim=-1)
        y_true.extend(labels.cpu().tolist())
        y_pred.extend(preds.cpu().tolist())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=label_names, digits=4))

    metrics = {
        "accuracy":  accuracy_score(y_true, y_pred),
        "f1":        f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall":    recall_score(y_true, y_pred, average="macro"),
    }
    print(
        f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
        f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}"
    )
    return metrics


class _EvalRobertaDS(Dataset):
    """Simple dataset for preprocessed texts and integer labels."""
    def __init__(self, texts, labels, max_len=128, backbone=BACKBONE):
        self.texts = list(texts)
        self.labels = list(labels)
        self.max_len = max_len
        self.tok = AutoTokenizer.from_pretrained(backbone, use_fast=True)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        enc = self.tok(self.texts[i], truncation=True, max_length=self.max_len)
        enc["labels"] = int(self.labels[i])
        return enc

# === 4) Evaluate with your existing loader ===
print("Evaluating INT8 with existing test_loader_clean...")
int8_metrics = eval_int8_cpu(q_model, test_loader_clean, label_names=ordered_labels)


Evaluating INT8 with existing test_loader_clean...
Classification Report:
                    precision    recall  f1-score   support

Extremely Negative     0.8943    0.8429    0.8678       592
          Negative     0.8023    0.8732    0.8362      1041
           Neutral     0.8946    0.8498    0.8716       619
          Positive     0.8128    0.7930    0.8028       947
Extremely Positive     0.8538    0.8481    0.8509       599

          accuracy                         0.8407      3798
         macro avg     0.8515    0.8414    0.8459      3798
      weighted avg     0.8424    0.8407    0.8409      3798

Accuracy=0.8407 | F1(macro)=0.8459 | Precision(macro)=0.8515 | Recall(macro)=0.8414


# unstructured pruning

In [ ]:
# prune_best_model2_roberta.py
import os
import tempfile
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn.utils import prune
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# If RobertaWithDropout is defined elsewhere, import it:
# from your_model_defs import RobertaWithDropout

# ======================
# Basic configuration
# ======================
MODEL_DIR  =data_dir
MODEL_FILE = "HF_best_model_stage3_YR.pt"
BACKBONE   = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS = 5
DROPOUT    = 0.2

PRUNE_AMOUNT         = 0.40  # 40% global unstructured L1
PRUNE_CLASSIFIER     = False # keep classifier dense by default
PRUNED_SD_SAVE_PATH  = os.path.join(MODEL_DIR, f"{Path(MODEL_FILE).stem}_pruned40_sd.pt")

# Use your existing labels or override here
ordered_labels = ["Extremely Negative","Negative","Neutral","Positive","Extremely Positive"]

# ======================
# Utilities
# ======================
def state_dict_size_mb(model_or_sd) -> float:
    sd = model_or_sd if isinstance(model_or_sd, dict) else model_or_sd.state_dict()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(sd, tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

def load_fp32_model_state(model_dir, model_file):
    """Load FP32 weights into the exact architecture wrapper."""
    path = os.path.join(model_dir, model_file)
    model = RobertaWithDropout(BACKBONE, num_labels=NUM_LABELS, dropout_rate=DROPOUT)
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval().cpu()
    return model

def collect_encoder_linear_params(model, prune_classifier=False):
    """
    Collect (module, 'weight') pairs for nn.Linear layers.
    By default, only prune Linear layers under 'roberta.encoder'.
    If prune_classifier=True, also include classifier Linear layers.
    """
    targets = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        is_encoder = name.startswith("roberta.encoder")
        is_classifier = name.startswith("classifier") or ".classifier." in name
        if is_encoder:
            targets.append((module, "weight"))
        elif prune_classifier and is_classifier:
            targets.append((module, "weight"))
    return targets

def apply_global_l1(model, amount=0.4, prune_classifier=False):
    """Apply global unstructured L1 pruning over selected Linear weights."""
    targets = collect_encoder_linear_params(model, prune_classifier=prune_classifier)
    if len(targets) == 0:
        raise RuntimeError("No Linear layers collected for pruning. Check your module naming/paths.")
    prune.global_unstructured(targets, pruning_method=prune.L1Unstructured, amount=amount)

    # Report density/sparsity on masked weights (viewed with the pruning mask)
    total, nonzero = 0, 0
    for m, _ in targets:
        w = m.weight  # masked view
        total   += w.numel()
        nonzero += (w != 0).sum().item()
    density  = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Applied global L1 pruning: amount={amount:.2f} | density={density:.2%} | sparsity={sparsity:.2%}")
    return targets, density, sparsity

def remove_pruning_masks(targets):
    """Make pruning permanent by removing mask reparameterizations."""
    for m, _ in targets:
        prune.remove(m, "weight")

@torch.inference_mode()
def evaluate_model_cpu(model, dataloader, label_names=None):
    """Generic CPU evaluation for classification models that return logits."""
    model.eval().cpu()
    y_true, y_pred = [], []
    for batch in dataloader:
        # Ensure tensors are on CPU
        for k, v in list(batch.items()):
            if isinstance(v, torch.Tensor):
                batch[k] = v.cpu()

        labels = batch.pop("labels") if "labels" in batch else batch.pop("label", None)
        outputs = model(**batch)

        # Robust logits extraction
        if hasattr(outputs, "logits"):
            logits = outputs.logits
        elif isinstance(outputs, dict) and "logits" in outputs:
            logits = outputs["logits"]
        elif isinstance(outputs, (list, tuple)):
            logits = outputs[0]
        else:
            logits = outputs

        preds = torch.argmax(logits, dim=-1)
        y_pred.extend(preds.cpu().tolist())
        if labels is not None:
            y_true.extend(labels.cpu().tolist())

    if len(y_true) > 0:
        print("Classification Report:")
        print(classification_report(y_true, y_pred, target_names=label_names, digits=4))
        metrics = {
            "accuracy":  accuracy_score(y_true, y_pred),
            "f1":        f1_score(y_true, y_pred, average="macro"),
            "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
            "recall":    recall_score(y_true, y_pred, average="macro"),
        }
        print(
            f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
            f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}"
        )
        return metrics
    else:
        print("Labels not found in dataloader batches; returned predictions only.")
        return {}

# ======================
# Run pruning workflow
# ======================
model = load_fp32_model_state(MODEL_DIR, MODEL_FILE)
print(f"Baseline state_dict size (dense): {state_dict_size_mb(model):.2f} MB")

targets, density, sparsity = apply_global_l1(
    model,
    amount=PRUNE_AMOUNT,
    prune_classifier=PRUNE_CLASSIFIER
)

# (Optional) short fine-tuning step to recover accuracy could be placed here.

remove_pruning_masks(targets)
print(f"Pruned state_dict size (dense save): {state_dict_size_mb(model):.2f} MB")

torch.save(model.state_dict(), PRUNED_SD_SAVE_PATH)
print("Saved pruned state_dict to:", PRUNED_SD_SAVE_PATH)

# ======================
# Evaluate with your existing loader
# ======================
# Assumes you already have: test_loader_clean
# If your loader is named differently, replace below.
if "test_loader_clean" in globals():
    print("Evaluating PRUNED model on test_loader_clean (CPU)...")
    pruned_metrics = evaluate_model_cpu(model, test_loader_clean, label_names=ordered_labels)
else:
    print("Skipping evaluation: `test_loader_clean` not found in globals().")


Baseline state_dict size (dense): 475.58 MB
Applied global L1 pruning: amount=0.40 | density=60.00% | sparsity=40.00%
Pruned state_dict size (dense save): 475.58 MB
Saved pruned state_dict to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3_YR_pruned40_sd.pt
Evaluating PRUNED model on test_loader_clean (CPU)...
Classification Report:
                    precision    recall  f1-score   support

Extremely Negative     0.9875    0.1334    0.2351       592
          Negative     0.6047    0.8742    0.7148      1041
           Neutral     0.8526    0.8691    0.8608       619
          Positive     0.5912    0.8933    0.7115       947
Extremely Positive     0.9868    0.2487    0.3973       599

          accuracy                         0.6640      3798
         macro avg     0.8045    0.6038    0.5839      3798
      weighted avg     0.7616    0.6640    0.6130      3798

Accuracy=0.6640 | F1(macro)=0.5839 | Precision(macro)=0.8045 | Recall(macro)=0.6038


In [ ]:
# prune_best_model2_roberta.py
import os
import tempfile
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn.utils import prune
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# If RobertaWithDropout is defined elsewhere, import it:
# from your_model_defs import RobertaWithDropout

# ======================
# Basic configuration
# ======================
MODEL_DIR  =data_dir
MODEL_FILE = "HF_best_model_stage3_YR.pt"
BACKBONE   = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS = 5
DROPOUT    = 0.2

PRUNE_AMOUNT         = 0.30  # 30% global unstructured L1
PRUNE_CLASSIFIER     = False # keep classifier dense by default
PRUNED_SD_SAVE_PATH  = os.path.join(MODEL_DIR, f"{Path(MODEL_FILE).stem}_pruned30_sd.pt")

# Use your existing labels or override here
ordered_labels = ["Extremely Negative","Negative","Neutral","Positive","Extremely Positive"]

# ======================
# Utilities
# ======================
def state_dict_size_mb(model_or_sd) -> float:
    sd = model_or_sd if isinstance(model_or_sd, dict) else model_or_sd.state_dict()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(sd, tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

def load_fp32_model_state(model_dir, model_file):
    """Load FP32 weights into the exact architecture wrapper."""
    path = os.path.join(model_dir, model_file)
    model = RobertaWithDropout(BACKBONE, num_labels=NUM_LABELS, dropout_rate=DROPOUT)
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval().cpu()
    return model

def collect_encoder_linear_params(model, prune_classifier=False):
    """
    Collect (module, 'weight') pairs for nn.Linear layers.
    By default, only prune Linear layers under 'roberta.encoder'.
    If prune_classifier=True, also include classifier Linear layers.
    """
    targets = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        is_encoder = name.startswith("roberta.encoder")
        is_classifier = name.startswith("classifier") or ".classifier." in name
        if is_encoder:
            targets.append((module, "weight"))
        elif prune_classifier and is_classifier:
            targets.append((module, "weight"))
    return targets

def apply_global_l1(model, amount=0.4, prune_classifier=False):
    """Apply global unstructured L1 pruning over selected Linear weights."""
    targets = collect_encoder_linear_params(model, prune_classifier=prune_classifier)
    if len(targets) == 0:
        raise RuntimeError("No Linear layers collected for pruning. Check your module naming/paths.")
    prune.global_unstructured(targets, pruning_method=prune.L1Unstructured, amount=amount)

    # Report density/sparsity on masked weights (viewed with the pruning mask)
    total, nonzero = 0, 0
    for m, _ in targets:
        w = m.weight  # masked view
        total   += w.numel()
        nonzero += (w != 0).sum().item()
    density  = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Applied global L1 pruning: amount={amount:.2f} | density={density:.2%} | sparsity={sparsity:.2%}")
    return targets, density, sparsity

def remove_pruning_masks(targets):
    """Make pruning permanent by removing mask reparameterizations."""
    for m, _ in targets:
        prune.remove(m, "weight")

@torch.inference_mode()
def evaluate_model_cpu(model, dataloader, label_names=None):
    """Generic CPU evaluation for classification models that return logits."""
    model.eval().cpu()
    y_true, y_pred = [], []
    for batch in dataloader:
        # Ensure tensors are on CPU
        for k, v in list(batch.items()):
            if isinstance(v, torch.Tensor):
                batch[k] = v.cpu()

        labels = batch.pop("labels") if "labels" in batch else batch.pop("label", None)
        outputs = model(**batch)

        # Robust logits extraction
        if hasattr(outputs, "logits"):
            logits = outputs.logits
        elif isinstance(outputs, dict) and "logits" in outputs:
            logits = outputs["logits"]
        elif isinstance(outputs, (list, tuple)):
            logits = outputs[0]
        else:
            logits = outputs

        preds = torch.argmax(logits, dim=-1)
        y_pred.extend(preds.cpu().tolist())
        if labels is not None:
            y_true.extend(labels.cpu().tolist())

    if len(y_true) > 0:
        print("Classification Report:")
        print(classification_report(y_true, y_pred, target_names=label_names, digits=4))
        metrics = {
            "accuracy":  accuracy_score(y_true, y_pred),
            "f1":        f1_score(y_true, y_pred, average="macro"),
            "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
            "recall":    recall_score(y_true, y_pred, average="macro"),
        }
        print(
            f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
            f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}"
        )
        return metrics
    else:
        print("Labels not found in dataloader batches; returned predictions only.")
        return {}

# ======================
# Run pruning workflow
# ======================
model = load_fp32_model_state(MODEL_DIR, MODEL_FILE)
print(f"Baseline state_dict size (dense): {state_dict_size_mb(model):.2f} MB")

targets, density, sparsity = apply_global_l1(
    model,
    amount=PRUNE_AMOUNT,
    prune_classifier=PRUNE_CLASSIFIER
)

# (Optional) short fine-tuning step to recover accuracy could be placed here.

remove_pruning_masks(targets)
print(f"Pruned state_dict size (dense save): {state_dict_size_mb(model):.2f} MB")

torch.save(model.state_dict(), PRUNED_SD_SAVE_PATH)
print("Saved pruned state_dict to:", PRUNED_SD_SAVE_PATH)

# ======================
# Evaluate with your existing loader
# ======================
# Assumes you already have: test_loader_clean
# If your loader is named differently, replace below.
if "test_loader_clean" in globals():
    print("Evaluating PRUNED model on test_loader_clean (CPU)...")
    pruned_metrics = evaluate_model_cpu(model, test_loader_clean, label_names=ordered_labels)
else:
    print("Skipping evaluation: `test_loader_clean` not found in globals().")


Baseline state_dict size (dense): 475.58 MB
Applied global L1 pruning: amount=0.30 | density=70.00% | sparsity=30.00%
Pruned state_dict size (dense save): 475.58 MB
Saved pruned state_dict to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3_YR_pruned30_sd.pt
Evaluating PRUNED model on test_loader_clean (CPU)...
Classification Report:
                    precision    recall  f1-score   support

Extremely Negative     0.9723    0.5338    0.6892       592
          Negative     0.7106    0.9059    0.7965      1041
           Neutral     0.8864    0.8449    0.8652       619
          Positive     0.7054    0.8849    0.7850       947
Extremely Positive     0.9538    0.5860    0.7260       599

          accuracy                         0.7823      3798
         macro avg     0.8457    0.7511    0.7724      3798
      weighted avg     0.8171    0.7823    0.7770      3798

Accuracy=0.7823 | F1(macro)=0.7724 | Precision(macro)=0.8457 | Recall(macro)=0.7511


In [ ]:
# replace collect_encoder_linear_params(...) with:
def collect_ffn_linear_params_only(model, prune_classifier=False):
    """
    Collect (name, module) for FFN Linear layers only:
    - ...intermediate.dense
    - ...output.dense
    Leaves attention (q/k/v/out_proj) untouched.
    """
    pairs = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        if name.startswith("roberta.encoder"):
            is_ffn = (".intermediate.dense" in name) or (".output.dense" in name)
            if is_ffn:
                pairs.append((name, module))
        elif prune_classifier and ("classifier" in name):
            pairs.append((name, module))
    return pairs

  # new: uniform per-layer L1 pruning on FFN only
def apply_uniform_l1_ffn(model, amount_per_layer=0.22, prune_classifier=False):
    pairs = collect_ffn_linear_params_only(model, prune_classifier=prune_classifier)
    if len(pairs) == 0:
        raise RuntimeError("No FFN Linear layers collected for pruning.")

    # apply same amount per layer
    for _, module in pairs:
        prune.l1_unstructured(module, name="weight", amount=amount_per_layer)

    # report density/sparsity
    total, nonzero = 0, 0
    for _, m in pairs:
        w = m.weight
        total   += w.numel()
        nonzero += (w != 0).sum().item()
    density  = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Applied FFN-only uniform L1: per-layer amount={amount_per_layer:.2f} | "
          f"density={density:.2%} | sparsity={sparsity:.2%}")

    # return targets in the (module,"weight") shape for remove()
    targets = [(m, "weight") for _, m in pairs]
    return targets, density, sparsity



In [ ]:
# prune_hf_stage3_YR_ffn_uniform.py
import os
import tempfile
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn.utils import prune
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# from your_model_defs import RobertaWithDropout

# ======================
# Basic configuration
# ======================
MODEL_DIR  =data_dir
MODEL_FILE = "HF_best_model_stage3_YR.pt"
BACKBONE   = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS = 5
DROPOUT    = 0.2

PRUNE_PER_LAYER      = 0.22   # try 0.20–0.25; tune to reach ≥80%
PRUNE_CLASSIFIER     = False  # keep classifier dense by default
PRUNED_SD_SAVE_PATH  = os.path.join(
    MODEL_DIR, f"{Path(MODEL_FILE).stem}_ffn_uniform{int(PRUNE_PER_LAYER*100)}_sd.pt"
)

ordered_labels = ["Extremely Negative","Negative","Neutral","Positive","Extremely Positive"]

# ======================
# Utilities
# ======================
def state_dict_size_mb(model_or_sd) -> float:
    sd = model_or_sd if isinstance(model_or_sd, dict) else model_or_sd.state_dict()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(sd, tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

def load_fp32_model_state(model_dir, model_file):
    """Load FP32 weights into the exact architecture wrapper."""
    path = os.path.join(model_dir, model_file)
    model = RobertaWithDropout(BACKBONE, num_labels=NUM_LABELS, dropout_rate=DROPOUT)
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval().cpu()
    return model

def collect_ffn_linear_params_only(model, prune_classifier=False):
    """
    Collect Linear weights ONLY from FFN blocks under roberta.encoder:
    - ...intermediate.dense
    - ...output.dense
    Leave attention (q/k/v/out_proj) untouched.
    """
    pairs = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        if name.startswith("roberta.encoder"):
            is_ffn = (".intermediate.dense" in name) or (".output.dense" in name)
            if is_ffn:
                pairs.append((name, module))
        elif prune_classifier and ("classifier" in name):
            pairs.append((name, module))
    return pairs

def apply_uniform_l1_ffn(model, amount_per_layer=0.22, prune_classifier=False):
    """Apply the SAME L1-unstructured amount per FFN Linear layer."""
    pairs = collect_ffn_linear_params_only(model, prune_classifier=prune_classifier)
    if len(pairs) == 0:
        raise RuntimeError("No FFN Linear layers collected for pruning.")

    for _, module in pairs:
        prune.l1_unstructured(module, name="weight", amount=amount_per_layer)

    # report overall density/sparsity on masked weights
    total, nonzero = 0, 0
    for _, m in pairs:
        w = m.weight
        total   += w.numel()
        nonzero += (w != 0).sum().item()
    density  = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Applied FFN-only uniform L1: per-layer amount={amount_per_layer:.2f} | "
          f"density={density:.2%} | sparsity={sparsity:.2%}")

    # return targets in (module, "weight") shape so we can remove() later
    targets = [(m, "weight") for _, m in pairs]
    return targets, density, sparsity

def remove_pruning_masks(targets):
    """Make pruning permanent by removing mask reparameterizations."""
    for m, _ in targets:
        prune.remove(m, "weight")

@torch.inference_mode()
def evaluate_model_cpu(model, dataloader, label_names=None):
    """Generic CPU evaluation for classification models that return logits."""
    model.eval().cpu()
    y_true, y_pred = [], []
    for batch in dataloader:
        # ensure tensors on CPU
        for k, v in list(batch.items()):
            if isinstance(v, torch.Tensor):
                batch[k] = v.cpu()

        labels = batch.pop("labels") if "labels" in batch else batch.pop("label", None)
        batch.pop("token_type_ids", None)  # RoBERTa doesn't use it

        outputs = model(**batch)
        logits = (outputs.logits if hasattr(outputs, "logits")
                  else outputs["logits"] if isinstance(outputs, dict)
                  else outputs[0])
        preds = torch.argmax(logits, dim=-1)
        y_pred.extend(preds.cpu().tolist())
        if labels is not None:
            y_true.extend(labels.cpu().tolist())

    if len(y_true) > 0:
        print("\n=== Classification Report ===")
        print(classification_report(y_true, y_pred, target_names=label_names, digits=4))
        metrics = {
            "accuracy":  accuracy_score(y_true, y_pred),
            "f1":        f1_score(y_true, y_pred, average="macro"),
            "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
            "recall":    recall_score(y_true, y_pred, average="macro"),
        }
        print(f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
              f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}")
        return metrics
    else:
        print("Labels not found in dataloader batches; returned predictions only.")
        return {}

# ======================
# Run pruning workflow
# ======================
model = load_fp32_model_state(MODEL_DIR, MODEL_FILE)
print(f"Baseline state_dict size (dense): {state_dict_size_mb(model):.2f} MB")

targets, density, sparsity = apply_uniform_l1_ffn(
    model, amount_per_layer=PRUNE_PER_LAYER, prune_classifier=PRUNE_CLASSIFIER
)

remove_pruning_masks(targets)
print(f"Pruned state_dict size (dense save): {state_dict_size_mb(model):.2f} MB")

torch.save(model.state_dict(), PRUNED_SD_SAVE_PATH)
print("Saved pruned state_dict to:", PRUNED_SD_SAVE_PATH)

# ======================
# Evaluate with your existing loader
# ======================
if "test_loader_clean" in globals():
    print("\nEvaluating PRUNED model on test_loader_clean (CPU)...")
    pruned_metrics = evaluate_model_cpu(model, test_loader_clean, label_names=ordered_labels)
    print("Test metrics:", pruned_metrics)
else:
    print("Skipping evaluation: `test_loader_clean` not found in globals().")


Baseline state_dict size (dense): 475.58 MB
Applied FFN-only uniform L1: per-layer amount=0.22 | density=78.00% | sparsity=22.00%
Pruned state_dict size (dense save): 475.58 MB
Saved pruned state_dict to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3_YR_ffn_uniform22_sd.pt

Evaluating PRUNED model on test_loader_clean (CPU)...

=== Classification Report ===
                    precision    recall  f1-score   support

Extremely Negative     0.9327    0.7720    0.8447       592
          Negative     0.7940    0.8886    0.8386      1041
           Neutral     0.8798    0.8514    0.8654       619
          Positive     0.7666    0.8532    0.8076       947
Extremely Positive     0.9041    0.7396    0.8136       599

          accuracy                         0.8320      3798
         macro avg     0.8554    0.8209    0.8340      3798
      weighted avg     0.8401    0.8320    0.8322      3798

Accuracy=0.8320 | F1(macro)=0.8340 | Precision(macro)=0.8554 | Recall(macro)=0.8209
T

# Knowledge Distillation Demo

In [ ]:
# ===== KD with RoBERTa teacher + DistilRoBERTa student (works with your TextClsDataset) =====
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback

device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------- Datasets (build from your DataFrames) ----------
NUM_LABELS = len(ORDERED_LABELS)

train_ds = TextClsDataset(train_df, text_col="ProcessedTweet", label_col="label")
val_ds   = TextClsDataset(eval_df,   text_col="ProcessedTweet", label_col="label")
test_ds  = TextClsDataset(test_df,   text_col="ProcessedTweet", label_col="label")  # for final report

# ---------- Teacher: RoBERTa (frozen) ----------
BACKBONE_TEACHER = "cardiffnlp/twitter-roberta-base-sentiment-latest"
TEACHER_CKPT     = data_dir /"HF_best_model_stage3_YR.pt"  # <-- your RoBERTa weights

def load_fp32_teacher_roberta(path, num_labels):
    # RobertaWithDropout must be defined (the CLS-based version)
    model = RobertaWithDropout(BACKBONE_TEACHER, num_labels=num_labels, dropout_rate=0.2)
    state = torch.load(path, map_location="cpu")  # expects state_dict
    model.load_state_dict(state, strict=True)
    model.eval()
    for p in model.parameters(): p.requires_grad = False
    return model

teacher = load_fp32_teacher_roberta(TEACHER_CKPT, NUM_LABELS).to(device)

# ---------- Student: DistilRoBERTa ----------
student = AutoModelForSequenceClassification.from_pretrained(
    "distilroberta-base",
    num_labels=NUM_LABELS
).to(device)

# ---------- Metrics (add micro/weighted for epoch table) ----------
def compute_metrics_fn(eval_pred):
    preds  = getattr(eval_pred, "predictions", eval_pred[0])
    labels = getattr(eval_pred, "label_ids",    eval_pred[1])
    if isinstance(preds, tuple): preds = preds[0]
    preds = np.argmax(preds, axis=-1)
    return {
        "accuracy":      accuracy_score(labels, preds),
        "f1_macro":      f1_score(labels, preds, average="macro"),
        "f1_micro":      f1_score(labels, preds, average="micro"),
        "f1_weighted":   f1_score(labels, preds, average="weighted"),
        "precision":     precision_score(labels, preds, average="macro", zero_division=0),
        "recall":        recall_score(labels, preds, average="macro"),
    }

# ---------- KD Trainer ----------
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.7, **kwargs):
        super().__init__(*args, **kwargs)
        assert teacher_model is not None, "teacher_model must be provided"
        self.teacher = teacher_model
        self.temperature = temperature
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # RoBERTa/DistilRoBERTa do not use token_type_ids
        s_inputs = dict(inputs)
        s_inputs.pop("token_type_ids", None)

        out_s = model(**s_inputs)
        logits_student = out_s.logits

        with torch.no_grad():
            t_out = self.teacher(
                input_ids=s_inputs["input_ids"],
                attention_mask=s_inputs.get("attention_mask", None)
            )
            logits_teacher = t_out["logits"] if isinstance(t_out, dict) else t_out.logits

        loss_ce = F.cross_entropy(logits_student, inputs["labels"])

        T = self.temperature
        loss_kl = F.kl_div(
            F.log_softmax(logits_student / T, dim=-1),
            F.softmax(logits_teacher / T, dim=-1),
            reduction="batchmean"
        ) * (T ** 2)

        loss = self.alpha * loss_ce + (1.0 - self.alpha) * loss_kl
        return (loss, out_s) if return_outputs else loss

# ---------- Pretty per-epoch table ----------
class EpochTableCallback(TrainerCallback):
    def __init__(self):
        self._printed_header = False
        self.rows = []

    def _last_train_loss_for_epoch(self, state, epoch_int):
        for rec in reversed(state.log_history):
            if rec.get("epoch") is None: continue
            if int(round(rec["epoch"])) == epoch_int and "loss" in rec:
                return float(rec["loss"])
        return float("nan")

    def on_epoch_end(self, args, state, control, **kwargs):
        control.should_evaluate = True  # force eval at each epoch end

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        from math import isnan
        ep = int(round(metrics.get("epoch", state.epoch or 0))) or (len(self.rows) + 1)
        tr_loss = self._last_train_loss_for_epoch(state, ep)
        row = {
            "Epoch": ep,
            "Training Loss": tr_loss,
            "Validation Loss": float(metrics.get("eval_loss", "nan")),
            "Accuracy": float(metrics.get("eval_accuracy", "nan")),
            "F1 Micro": float(metrics.get("eval_f1_micro", "nan")),
            "F1 Macro": float(metrics.get("eval_f1_macro", "nan")),
            "F1 Weighted": float(metrics.get("eval_f1_weighted", "nan")),
        }
        self.rows.append(row)
        if not self._printed_header:
            print("\nEpoch  Training Loss  Validation Loss   Accuracy   F1 Micro   F1 Macro  F1 Weighted")
            self._printed_header = True
        def f(x):
            from math import isnan
            return "   nan" if (x != x) or isnan(x) else f"{x:10.6f}"
        print(f"{row['Epoch']:>5}  {f(row['Training Loss'])}   {f(row['Validation Loss'])}  "
              f"{f(row['Accuracy'])}  {f(row['F1 Micro'])}  {f(row['F1 Macro'])}  {f(row['F1 Weighted'])}")

epoch_table = EpochTableCallback()

# ---------- TrainingArguments (minimal; callback handles per-epoch eval) ----------
training_args = TrainingArguments(
    output_dir="./distill_results_stage3_YR_student",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none",
    remove_unused_columns=False,  # keep dict keys from our dataset
)

# ---------- Trainer ----------
trainer_distill = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics_fn,
    data_collator=collator,  # your collator (RoBERTa)
)

trainer_distill.add_callback(epoch_table)

# ---------- Train ----------
trainer_distill.train()

# ---------- End-of-run multi-line Classification Report on TEST ----------
pred = trainer_distill.predict(test_ds)  # uses the same collator internally
y_true = pred.label_ids
y_pred = pred.predictions.argmax(axis=-1)

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=ORDERED_LABELS, digits=4))

# ---------- Save student ----------
trainer_distill.save_model("./distill_results_stage3_YR_student/best_student")
print("\nDistillation complete. Student (DistilRoBERTa) model saved.")
print("Student parameter count:", sum(p.numel() for p in student.parameters()))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch  Training Loss  Validation Loss   Accuracy   F1 Micro   F1 Macro  F1 Weighted
    1    0.973300     0.816348    0.769679    0.769679    0.779466    0.769256
    2    0.666600     0.661543    0.819485    0.819485    0.824585    0.819114
    3    0.445800     0.672272    0.813897    0.813897    0.819562    0.812868
    4    0.334600     0.739040    0.818270    0.818270    0.823273    0.817714
    5    0.271900     0.609968    0.846453    0.846453    0.850710    0.846310



=== Classification Report ===
                    precision    recall  f1-score   support

Extremely Negative     0.7956    0.9071    0.8477       592
          Negative     0.8173    0.7819    0.7992      1041
           Neutral     0.9286    0.8191    0.8704       619
          Positive     0.8034    0.7941    0.7987       947
Extremely Positive     0.8202    0.8831    0.8505       599

          accuracy                         0.8265      3798
         macro avg     0.8330    0.8371    0.8333      3798
      weighted avg     0.8290    0.8265    0.8263      3798


Distillation complete. Student (DistilRoBERTa) model saved.
Student parameter count: 82122245
